In [10]:
import numpy as np
import pandas as pd
import json
from datetime import datetime
import pickle 

In [11]:
data = pd.read_csv('../data/egrul.csv')
data.head(5)

,id,inn,data,fetched_at
0,1,1001028755,"{""data"": {""ИНН"": ""1001028755"", ""КПП"": ""7806010...",2025-04-21 13:02:51.38553
1,2,1001001390,"{""data"": {""ИНН"": ""1001001390"", ""КПП"": ""1001010...",2025-04-21 13:02:51.398212
2,3,1001098784,"{""data"": {""ИНН"": ""1001098784"", ""КПП"": ""1001010...",2025-04-21 13:02:51.431791
3,4,1001095007,"{""data"": {""ИНН"": ""1001095007"", ""КПП"": ""1001010...",2025-04-21 13:02:51.439663
4,5,1001004095,"{""data"": {""ИНН"": ""1001004095"", ""КПП"": ""1001010...",2025-04-21 13:02:51.489868


In [12]:
data['data'] = data['data'].apply(json.loads)
df_expanded = pd.json_normalize(data['data'])
df_expanded.columns = [col.replace('.', '_') for col in df_expanded.columns]

In [28]:
features = pd.DataFrame()

features['ИНН'] = df_expanded['data_ИНН'].astype('int64')

features['ДатаРег'] = pd.to_datetime(df_expanded['data_ДатаРег'], errors='coerce')
features['Возраст_компании'] = ((datetime.now() - features['ДатаРег']).dt.days // 365).fillna(-1)

features['Статус'] = df_expanded['data_Статус_Наим'].fillna('data_Неизвестно')

features['Недостоверный_адрес'] = df_expanded['data_ЮрАдрес_Недост'].fillna(False)

features['Сумма_налоговой_недоимки'] = df_expanded['data_Налоги_СумНедоим'].fillna(0).astype(float)

features['Сумма_уплаченных_налогов'] = df_expanded['data_Налоги_СумУпл'].fillna(0).astype(float)

features['Особый_налоговый_режим'] = df_expanded['data_Налоги_ОсобРежим'].apply(lambda x: 1 if isinstance(x, list) and len(x) > 0 else 0)

features['Массовый_руководитель'] = df_expanded['data_МассРуковод'].fillna(False)

features['Дисквалифицированный_руководитель'] = df_expanded['data_ДисквЛица'].fillna(False)

features['Недостоверный_руководитель'] = df_expanded['data_Руковод'].apply(
    lambda x: any(d.get('Недост', False) for d in x) if isinstance(x, list) else False
)

features['Массовый_учредитель'] = df_expanded['data_МассУчред'].fillna(False)

features['Есть_банкротство'] = df_expanded['data_ЕФРСБ'].apply(lambda x: 1 if isinstance(x, list) and len(x) > 0 else 0)

features['Недобросовестный_поставщик'] = df_expanded['data_НедобПост'].fillna(False)

features['Под_санкциями'] = df_expanded['data_Санкции'].fillna(False)

features['Категория_МСП'] = df_expanded['data_РМСП_Кат'].fillna('Неизвестно')

features['Среднесписочная_численность'] = df_expanded['data_СЧР'].fillna(0)

with open('../data/egrul_pd.pkl', 'wb') as f: 
    pickle.dump(features, f)

features.head()


,ИНН,ДатаРег,Возраст_компании,Статус,Недостоверный_адрес,Сумма_налоговой_недоимки,Сумма_уплаченных_налогов,Особый_налоговый_режим,Массовый_руководитель,Дисквалифицированный_руководитель,Недостоверный_руководитель,Массовый_учредитель,Есть_банкротство,Недобросовестный_поставщик,Под_санкциями,Категория_МСП,Среднесписочная_численность
0,1001028755,2011-06-16,13,Действует,False,0.000000e+00,4.668453e+06,0,False,False,False,True,0,False,False,МАЛОЕ ПРЕДПРИЯТИЕ,39
1,1001001390,1994-09-14,30,Действует,False,2.072310e+07,2.478546e+08,0,False,False,False,False,0,False,False,Неизвестно,163
2,1001098784,2000-04-21,25,Юридическое лицо признано несостоятельным (бан...,False,1.269584e+08,1.331751e+07,0,False,False,False,False,1,False,False,МАЛОЕ ПРЕДПРИЯТИЕ,4
3,1001095007,1999-10-19,25,Действует,False,0.000000e+00,5.535989e+06,0,False,False,False,False,0,False,False,МАЛОЕ ПРЕДПРИЯТИЕ,3
4,1001004095,1993-02-25,32,Действует,False,0.000000e+00,1.695939e+07,0,False,False,False,False,0,False,False,МАЛОЕ ПРЕДПРИЯТИЕ,37


In [29]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 17 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   ИНН                                15000 non-null  int64         
 1   ДатаРег                            15000 non-null  datetime64[ns]
 2   Возраст_компании                   15000 non-null  int64         
 3   Статус                             15000 non-null  object        
 4   Недостоверный_адрес                15000 non-null  bool          
 5   Сумма_налоговой_недоимки           15000 non-null  float64       
 6   Сумма_уплаченных_налогов           15000 non-null  float64       
 7   Особый_налоговый_режим             15000 non-null  int64         
 8   Массовый_руководитель              15000 non-null  bool          
 9   Дисквалифицированный_руководитель  15000 non-null  bool          
 10  Недостоверный_руководитель        

In [18]:
features.isnull().sum()

ИНН                                  0
ДатаРег                              0
Возраст_компании                     0
Статус                               0
Недостоверный_адрес                  0
Сумма_налоговой_недоимки             0
Сумма_уплаченных_налогов             0
Особый_налоговый_режим               0
Массовый_руководитель                0
Дисквалифицированный_руководитель    0
Недостоверный_руководитель           0
Массовый_учредитель                  0
Есть_банкротство                     0
Недобросовестный_поставщик           0
Под_санкциями                        0
Категория_МСП                        0
Среднесписочная_численность          0
dtype: int64